In [8]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from statsmodels.distributions.empirical_distribution import ECDF


In [9]:
# load the data
path = "StudEmo_2022_sample"
train = pd.read_excel(path + "/train.xlsx")
test = pd.read_excel(path + "/test.xlsx")
validation = pd.read_excel(path + "/valid.xlsx")

df = pd.concat([train, test, validation])

In [10]:
# need to describe the data
df.columns
df.min()
df.max()

numerical_col = [
    "joy_x",
    "trust_x",
    "anticipation_x",
    "surprise_x",
    "fear_x",
    "sadness_x",
    "disgust_x",
    "anger_x",
    "valence_x",
    "arousal_x",
    "joy_y",
    "trust_y",
    "anticipation_y",
    "surprise_y",
    "fear_y",
    "sadness_y",
    "disgust_y",
    "anger_y",
    "valence_y",
    "arousal_y",
]

### Functions

#### ksdensity

In [4]:
def create_histogram(sample, which_col, person):
    """sample - one column
    histogram + median + mean
    person = { X , Y}
    which_col = {joy, trust, anticipation, ...}"""
    plt.figure(0)

    result = plt.hist(
        sample, bins=40, color="mediumseagreen", edgecolor="k", alpha=0.65
    )
    plt.axvline(sample.mean(), color="k", linestyle="dashed", linewidth=1)
    plt.axvline(sample.median(), color="k", linestyle="dashed", linewidth=1)

    min_ylim, max_ylim = plt.ylim()
    plt.text(
        sample.mean() * 1.1,
        max_ylim * 0.9,
        "Mean: {:.2f}".format(sample.mean()),
    )
    plt.text(
        sample.median() * 1.1,
        max_ylim * 0.8,
        "Median: {:.2f}".format(sample.median()),
    )

    plt.grid()
    plt.title(
        "Histogram of {which_col} for a person {person}".format(
            which_col=which_col, person=person
        ),
        loc="center",
    )
    plt.xlabel("Value of opinion")
    plt.ylabel("Frequency")
    plt.show()

#### ecdf

In [5]:
def create_ecdf(sample, which_col, person):
    """sample - one column
    person = {X , Y}
    which_col = {joy, trust, anticipation, ...}"""
    plt.figure(0)
    ecdf = ECDF(sample)
    plt.plot(ecdf.x, ecdf.y)

    plt.grid()
    plt.title(
        "Cumulative distribution function of {which_col} for a person {person}".format(
            which_col=which_col, person=person
        ),
        loc="center",
    )
    plt.xlabel("Sample")
    plt.ylabel("Probability")
    plt.show()

boxplot jakies slabe to na tych danych, nie wiem czy uzyjemy. Zobaczymy ile slajdow sie naprodukuje

In [11]:
def normalization(df, numerical_col):
    # normalization  musi byc min-max a nie -mean/ std bo mamy wartosci ujemne
    # w kolumnach
    normalized_df = (df[numerical_col] - df[numerical_col].min()) / (
        df[numerical_col].max() - df[numerical_col].min()
    )
    return normalized_df

In [1]:
def create_boxplot(normalized_df, numerical_col):
    plt.figure(figsize=(16, 5))
    plt.title("Boxplot of min-max normalized datasets", fontsize=14)
    boxplot = normalized_df.boxplot(column=numerical_col, rot=45, fontsize=12)

#### heat map

In [9]:
def create_heatmap(df, person):
    plt.subplots(figsize=(18, 12))
    corr = df.corr()
    sns.heatmap(
        corr,
        xticklabels=corr.columns.values,
        yticklabels=corr.columns.values,
        linewidths=0.5,
        annot=True,
    ).set(title="Heat map for a person {person}".format(person=person))

#### Analysis for person X

In [ ]:
for col_name in numerical_col[: len(numerical_col) // 2]:
    create_histogram(df[col_name], col_name[: len(col_name) - 2], "X")
    create_ecdf(df[col_name], col_name[: len(col_name) - 2], "X")
create_heatmap(df, "X")

#### Analysis for person Y

In [17]:
for col_name in numerical_col[len(numerical_col) // 2 + 1 :]:
    create_histogram(df[col_name], col_name[: len(col_name) - 2], "Y")
    create_ecdf(df[col_name], col_name[: len(col_name) - 2], "Y")
create_heatmap(df, "Y")

NameError: name 'create_histogram' is not defined

#### Comparison of X and Y; mean opinion value
calculate mean value

In [13]:
joy_x = numerical_col[0]
joy_y = numerical_col[10]
new_col = (df[joy_x] + df[joy_y]) / 2
mean_opinion = new_col
numerical = joy_x
names = [numerical[: len(numerical) - 2]]

for i in range(1, 10):
    new_col = (df[numerical_col[i]] + df[numerical_col[i + 10]]) / 2
    mean_opinion = pd.concat([mean_opinion, new_col], axis=1)

    column_name = numerical_col[i]
    names.append(column_name[: len(column_name) - 2])

mean_opinion.columns = names


### analysis

In [ ]:
for col_name in names:
    create_histogram(mean_opinion[col_name], col_name, "mean(X,Y)")
    create_ecdf(mean_opinion[col_name], col_name, "mean(X,Y)")
create_heatmap(mean_opinion, "mean(X,Y)")


In [ ]:
plt.hist(df["joy_x"], bins=10, alpha=0.5, label="joy_x")
plt.hist(df["joy_y"], bins=10, alpha=0.5, label="joy_y")
plt.xlabel("Data", size=14)
plt.legend(loc="upper right")